# Unity ML Agents
## Proximal Policy Optimization (PPO)
Contains an implementation of PPO as described [here](https://arxiv.org/abs/1707.06347).

In [4]:
import numpy as np
import os
import tensorflow as tf

from ppo.history import *
from ppo.models import *
from ppo.trainer import Trainer
from unityagents import *

### Hyperparameters

In [16]:
### General parameters
max_steps =  1.1e7 # Set maximum number of steps to run environment.
run_path = "ballcatcher3/Run4" # The sub-directory name for model and summary statistics
load_model = False # Whether to load a saved model.
train_model = True # Whether to train the model.
summary_freq = 10000 # Frequency at which to save training statistics.
save_freq = 50000 # Frequency at which to save model.
env_name = "ballcatcher3" # Name of the training environment file.
curriculum_file = None

### Algorithm-specific parameters for tuning
gamma = 0.99 # Reward discount rate.
lambd = 0.95 # Lambda parameter for GAE.
time_horizon = 100 # How many steps to collect per agent before adding to buffer.
beta = 1e-3 # Strength of entropy regularization
num_epoch = 4 # Number of gradient descent steps per batch of experiences.
num_layers = 2 # Number of hidden layers between state/observation encoding and value/policy layers.
epsilon = 0.15 # Acceptable threshold around ratio of old and new policy probabilities.
buffer_size = 20480 # How large the experience buffer should be before gradient descent.
learning_rate = 1e-5 # Model learning rate.
hidden_units = 96 # Number of units in hidden layer.
batch_size = 128 # How many experiences per gradient descent update step.
normalize = False

# num_layers = 1
# hidden_units = 10
# time_horizon = 1000
# learning_rate = 1e-3, 1e-4, 1e-5
# normalize = True

# could try:
# num_epoch = 2
# beta = 0

### Logging dictionary for hyperparameters
hyperparameter_dict = {'max_steps':max_steps, 'run_path':run_path, 'env_name':env_name,
    'curriculum_file':curriculum_file, 'gamma':gamma, 'lambd':lambd, 'time_horizon':time_horizon,
    'beta':beta, 'num_epoch':num_epoch, 'epsilon':epsilon, 'buffe_size':buffer_size,
    'leaning_rate':learning_rate, 'hidden_units':hidden_units, 'batch_size':batch_size}

### Load the environment

In [17]:
env = UnityEnvironment(file_name=env_name, curriculum=curriculum_file,worker_id=4,base_port=5009)
print(str(env))
brain_name = env.external_brain_names[0]

INFO:unityagents:
'CatcherAcademy' started successfully!


Unity Academy name: CatcherAcademy
        Number of brains: 1
        Reset Parameters :
		
Unity brain name: Brain
        Number of observations (per agent): 0
        State space type: continuous
        State space size (per agent): 16
        Action space type: discrete
        Action space size (per agent): 4
        Memory space size (per agent): 24
        Action descriptions: , , , 


### Train the Agent(s)

In [18]:
tf.reset_default_graph()

if curriculum_file == "None":
    curriculum_file = None


def get_progress():
    if curriculum_file is not None:
        if env._curriculum.measure_type == "progress":
            return steps / max_steps
        elif env._curriculum.measure_type == "reward":
            return last_reward
        else:
            return None
    else:
        return None

# Create the Tensorflow model graph
ppo_model = create_agent_model(env, lr=learning_rate,
                               h_size=hidden_units, epsilon=epsilon,
                               beta=beta, max_step=max_steps, 
                               normalize=normalize, num_layers=num_layers)

is_continuous = (env.brains[brain_name].action_space_type == "continuous")
use_observations = (env.brains[brain_name].number_observations > 0)
use_states = (env.brains[brain_name].state_space_size > 0)

model_path = './models/ppo/{}'.format(run_path)
summary_path = './summaries/{}'.format(run_path)

if not os.path.exists(model_path):
    os.makedirs(model_path)

if not os.path.exists(summary_path):
    os.makedirs(summary_path)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    # Instantiate model parameters
    if load_model:
        print('Loading Model...')
        ckpt = tf.train.get_checkpoint_state(model_path)
        saver.restore(sess, ckpt.model_checkpoint_path)
    else:
        sess.run(init)
    steps, last_reward = sess.run([ppo_model.global_step, ppo_model.last_reward])    
    summary_writer = tf.summary.FileWriter(summary_path)
    info = env.reset(train_mode=train_model, progress=get_progress())[brain_name]
    trainer = Trainer(ppo_model, sess, info, is_continuous, use_observations, use_states, train_model)
    if train_model:
        trainer.write_text(summary_writer, 'Hyperparameters', hyperparameter_dict, steps)
    while steps <= max_steps:
        if env.global_done:
            info = env.reset(train_mode=train_model, progress=get_progress())[brain_name]
        # Decide and take an action
        new_info = trainer.take_action(info, env, brain_name, steps, normalize)
        info = new_info
        trainer.process_experiences(info, time_horizon, gamma, lambd)
        if len(trainer.training_buffer['actions']) > buffer_size and train_model:
            # Perform gradient descent with experience buffer
            trainer.update_model(batch_size, num_epoch)
        if steps % summary_freq == 0 and steps != 0 and train_model:
            # Write training statistics to tensorboard.
            trainer.write_summary(summary_writer, steps, env._curriculum.lesson_number)
        if steps % save_freq == 0 and steps != 0 and train_model:
            # Save Tensorflow model
            save_model(sess, model_path=model_path, steps=steps, saver=saver)
        steps += 1
        sess.run(ppo_model.increment_step)
        if len(trainer.stats['cumulative_reward']) > 0:
            mean_reward = np.mean(trainer.stats['cumulative_reward'])
            sess.run(ppo_model.update_reward, feed_dict={ppo_model.new_reward: mean_reward})
            last_reward = sess.run(ppo_model.last_reward)
    # Final save Tensorflow model
    if steps != 0 and train_model:
        save_model(sess, model_path=model_path, steps=steps, saver=saver)
env.close()
export_graph(model_path, env_name)

Step: 10000. Mean Reward: -25.871764269359982. Std of Reward: 6.895576667644034.
Step: 20000. Mean Reward: -26.215395162254293. Std of Reward: 13.433948457543913.
Step: 30000. Mean Reward: -20.656354106166685. Std of Reward: 21.639968079226684.
Step: 40000. Mean Reward: -23.753837124465285. Std of Reward: 20.251614216236035.
Step: 50000. Mean Reward: -16.621398162237465. Std of Reward: 25.316992150201603.
Saved Model
Step: 60000. Mean Reward: -13.95938583705287. Std of Reward: 26.138815425497253.
Step: 70000. Mean Reward: -8.033079626260903. Std of Reward: 26.671862850913566.
Step: 80000. Mean Reward: -6.25707615498313. Std of Reward: 27.52745696505806.
Step: 90000. Mean Reward: -1.0055950770160298. Std of Reward: 24.732343974523822.
Step: 100000. Mean Reward: 1.0991365573814968. Std of Reward: 24.720188217866927.
Saved Model
Step: 110000. Mean Reward: -4.126249864592809. Std of Reward: 26.60331805633026.
Step: 120000. Mean Reward: 1.0567074616584502. Std of Reward: 24.963451824966636.

Step: 1000000. Mean Reward: 24.807688622686133. Std of Reward: 5.726366692107065.
Saved Model
Step: 1010000. Mean Reward: 23.846339457446035. Std of Reward: 9.052075206076283.
Step: 1020000. Mean Reward: 24.633311524485247. Std of Reward: 5.5792956295574445.
Step: 1030000. Mean Reward: 24.466849128350237. Std of Reward: 6.6522778248465055.
Step: 1040000. Mean Reward: 24.919018547757453. Std of Reward: 4.0874435618459595.
Step: 1050000. Mean Reward: 24.74792303777007. Std of Reward: 4.480822913176565.
Saved Model
Step: 1060000. Mean Reward: 24.11559599311562. Std of Reward: 7.710909254237402.
Step: 1070000. Mean Reward: 24.95741972715663. Std of Reward: 4.276037706533414.
Step: 1080000. Mean Reward: 24.859974736647782. Std of Reward: 3.4228790050727684.
Step: 1090000. Mean Reward: 24.713693779405098. Std of Reward: 4.668712893459659.
Step: 1100000. Mean Reward: 24.92757838646914. Std of Reward: 2.5270201254859526.
Saved Model
Step: 1110000. Mean Reward: 24.77508436453969. Std of Reward:

KeyboardInterrupt: 

### Export the trained Tensorflow graph
Once the model has been trained and saved, we can export it as a .bytes file which Unity can embed.

In [19]:
export_graph(model_path, env_name)

INFO:tensorflow:Restoring parameters from ./models/ppo/ballcatcher3/Run4\model-1100000.cptk


INFO:tensorflow:Restoring parameters from ./models/ppo/ballcatcher3/Run4\model-1100000.cptk


INFO:tensorflow:Froze 4 variables.


INFO:tensorflow:Froze 4 variables.


Converted 4 variables to const ops.
